In [1]:
import json
import pandas as pd
import datetime
from tqdm import tqdm
from requests import get
import time
from pprint import pprint

format = "%Y-%m-%dT%H:%M:%S.000Z"
origin = datetime.datetime.strptime('2020-11-09T05:00:00.000Z', format)

file="covid-19-vaccination-stance-3249.csv"
given = {str(int(i["id"])): int(i["category"]) for i in pd.read_csv(file).T.to_dict().values()}

with open("twitter_keys.json") as f:
    header = {"Authorization": f"Bearer {json.load(f)['Bearer Token']}"}
# def ret_tweet(id):
#     return get(f"https://api.twitter.com/2/tweets/{id}?tweet.fields=created_at%2Cid%2Ctext%2Centities",
#                headers=header
#                ).json()

In [ ]:
a, b = get_row(1325774333504073728)
pprint(b)
print("\n\n")
pprint(a)

In [2]:
def get_row(id):
    if type(id)!=str:
        id = str(int(id))
    res = get(f"https://api.twitter.com/2/tweets/{id}?tweet.fields=created_at%2Cid%2Ctext%2Centities", headers=header).json()
    # time.sleep(0.5)
    if "errors" in res: return None
    elif "data" in res:
        res = res["data"]
        try: hashtags = [i["tag"] for i in res["entities"]["hashtags"]]
        except KeyError: hashtags = []
        return [{"date": round((datetime.datetime.strptime(res["created_at"], format)-origin).total_seconds()/(2.6*10**6), 6),
                 "text": res["text"],
                 "entities": res["entities"] if "entities" in res else {},
                 "y": given[id],
                },
                {"date": round((datetime.datetime.strptime(res["created_at"], format)-origin).total_seconds()/(2.6*10**6), 6),
                 "text": res["text"],
                 "entities": hashtags,
                 "y": given[id],
                }
            ]
    elif "title" in res and res["title"] == 'Too Many Requests':
        wait = 10
        till = datetime.datetime.now()+datetime.timedelta(minutes=wait)
        print(f"Too Many Requests, waiting {wait} minutes till {till.hour%12}:{till.minute}:{till.second}... ")
        time.sleep(60*wait)
        return get_row(id)
    else:
        return {"spl": res}

In [3]:
data = {}
dataE = {}
notFound = []
specialCase = {}
i=0
for x in tqdm(given.keys()):
    i+=1
    ID = str(int(x))
    res = get_row(ID)
    if res:
        if "spl" in res:
            specialCase[ID] = res
        else:
            dataE[ID], data[ID] = res
    else:
        notFound.append(ID)

 27%|██▋       | 861/3249 [06:58<19:24,  2.05it/s]

Too Many Requests, waiting 4 minutes till 9:21:55... 
Too Many Requests, waiting 4 minutes till 9:25:56... 
Too Many Requests, waiting 4 minutes till 9:29:56... 


 42%|████▏     | 1362/3249 [23:03<14:39,  2.15it/s]    

Too Many Requests, waiting 4 minutes till 9:38:0... 
Too Many Requests, waiting 4 minutes till 9:42:0... 


 42%|████▏     | 1363/3249 [31:04<75:48:51, 144.71s/it]

...

 88%|████████▊ | 2875/3249 [1:10:45<02:50,  2.19it/s]    

Too Many Requests, waiting 4 minutes till 10:25:42... 
Too Many Requests, waiting 4 minutes till 10:29:42... 


 89%|████████▊ | 2876/3249 [1:18:46<14:59:45, 144.73s/it]

Too Many Requests, waiting 4 minutes till 10:33:43... 


100%|██████████| 3249/3249 [2:07:06<00:00,  2.35s/it]    


In [4]:
with open("dataH.json", "w") as f:
    json.dump(data, f)
with open("dataHE.json", "w") as f:
    json.dump(dataE, f)

In [ ]:
print(len(data))

In [ ]:
dataE